In [1]:
import numpy as np
import math
import random
import os
import pybullet as p
import pybullet_data
import gym
from gym import error, spaces, utils
from gym.utils import seeding
import matplotlib.pyplot as plt
import time
import cv2

In [2]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, Dropout, concatenate
from keras.layers import Concatenate,Conv2D,BatchNormalization,MaxPooling2D
from keras.optimizers import Adam

C:\Users\SERVER\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\SERVER\Anaconda3\lib\site-packages\tensorflow\python\framew

In [3]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import  SequentialMemory
from rl.processors import Processor
from rl.callbacks import Callback

In [4]:
class BikeEnv(gym.Env):
    
    def __init__(self):
        p.connect(p.GUI)
        p.setRealTimeSimulation(1)
        p.resetDebugVisualizerCamera(cameraDistance=10, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55,-0.35,0.2])
        self.action_space = spaces.Discrete(6)
        self.observation_space = spaces.Dict({
            'sensor' : spaces.Box(np.array([-1000]*10), np.array([1000]*10)),
            'image'  : spaces.Box(low=0, high=1, shape=(64, 64))
        }) 
        
        self.timestep = 1./240.
        self.stp = 0
        
    def step(self, action):
        #print("i=%d" % self.stp)
        self.stp+=1
        #print("time = " , time.time()-self.time)
     
        if (action == 0):
            self.speed = self.speed + 1
        if (action == 1):
            self.speed = self.speed - 1 
        if (action == 2):
            self.speed = self.speed  
        if (action == 3):
            self.steer = self.steer - 1 
        if (action == 4):
            self.steer = self.steer + 1
        if (action == 5):
            self.steer = self.steer 
            
              
        self.applyAction([self.speed,self.steer])

        #p.setRealTimeSimulation(1)
        time.sleep(0.5)
        #p.setRealTimeSimulation(0)
        
        state = p.getLinkState(self.pid,0)[0]
        if state[2] <= 0.5 or  state[2] >= 2 or abs(self.speed)>2 or abs(self.steer)>4:
            reward = -100
            done = True
        else :
            #reward = math.sqrt((self.origin[0]-state[0])**2+(self.origin[1]-state[1])**2)
            reward = state[0] - self.origin[0]
            #reward = 1
            done = False
        self.origin = state 
        
        velocity = p.getBaseVelocity(self.pid)
        img = self.getImage()
        observation = {'sensor':list(self.getObservation()) + list(velocity[0])+list(velocity[1]),
                       'image': img}
        
        info = {'x':'','y':'','z':''}
        #print("Step: ",self.stp)
        #xx = time.time()
        #print("Time: ",xx-self.tttt)
        #self.tttt = xx
        #print("Action: ",action)
        #print("Reward: ",reward)
        #self.stp +=1
        return observation, reward, done, info
            
    def applyAction(self, motorCommands):
        targetVelocity = motorCommands[0] * self.speedMultiplier
        #print("targetVelocity")
        #print(targetVelocity)
        steeringAngle = motorCommands[1] * self.steeringMultiplier
        #print("steeringAngle")
        #print(steeringAngle)


        for motor in self.motorizedwheels:
            p.setJointMotorControl2(self.pid,
                                    motor,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=targetVelocity,
                                    force=self.maxForce)
        for steer in self.steeringLinks:
            p.setJointMotorControl2(self.pid,
                                    steer,
                                    p.POSITION_CONTROL,
                                    targetPosition=steeringAngle)

    def reset(self):
        #print("Reset")
        #print("setp:",self.stp)
        self.stp = 0

        p.resetSimulation()

        urdfRootPath = pybullet_data.getDataPath()
        planeUid = p.loadURDF(os.path.join(urdfRootPath,"plane.urdf"), basePosition=[0,0,0])
        
        
        for i in range(np.random.randint(1,10)):
            p.loadURDF(os.path.join(urdfRootPath, "sphere2.urdf"),basePosition=[
                np.random.randint(5,15),
                np.random.randint(-2,2),
               0.5
           ])
     
        self.pid = p.loadURDF(os.path.join(urdfRootPath, "bicycle/bike.urdf"),basePosition=[0,0,1])     
           
        
        self.origin = p.getLinkState(self.pid,0)[0]
        p.setGravity(0,0,-10)
        for wheel in range(p.getNumJoints(self.pid)):
            p.setJointMotorControl2(self.pid,
                                    wheel,
                                    p.VELOCITY_CONTROL,
                                    targetVelocity=0,
                                    force=0)

        self.steeringLinks = [0]
        self.maxForce = 20
        self.nMotors = 2
        self.motorizedwheels = [1, 2]
        self.speedMultiplier = 10.
        self.steeringMultiplier = 0.5
        
        self.speed = 0 
        self.steer = 0

        velocity = p.getBaseVelocity(self.pid)
        img = self.getImage()
        observation = {'sensor': list(self.getObservation()) + list(velocity[0])+list(velocity[1]),
                       'image': img}
        

        p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
        
        return observation
        
    
    def getImage(self):
        img = env.render()
        img = cv2.resize(img, (64,64), interpolation = cv2.INTER_NEAREST )
        img = img[:,:,2]
        img =img.reshape((64,64))
        img = np.asarray(img, dtype='float32')
        img /= 255.0
        return img
        
    def getObservationDimension(self):
        return len(self.getObservation())
    
    def getObservation(self):
        observation = []
        pos, orn = p.getBasePositionAndOrientation(self.pid)

        #observation.extend(list(pos))
        observation.extend(list(orn))
        return observation
        
    def render(self, mode='rgb_array'):
        pos, orn = p.getBasePositionAndOrientation(self.pid)
      
        view_matrix = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[pos[0]+11.3,pos[1],pos[2]],
                                                            distance=10,
                                                            yaw=-90 ,
                                                            pitch=0,
                                                            roll=0,
                                                            upAxisIndex=2)
        proj_matrix = p.computeProjectionMatrixFOV(fov=60,
                                                     aspect=float(960) /720,
                                                     nearVal=0.1,
                                                     farVal=100.0)
        (_, _, px, _, _) = p.getCameraImage(width=64,
                                              height=64,
                                              viewMatrix=view_matrix,
                                              projectionMatrix=proj_matrix,
                                              renderer=p.ER_BULLET_HARDWARE_OPENGL)

        rgb_array = np.array(px, dtype=np.uint8)
        rgb_array = np.reshape(rgb_array, (64,64, 4))

        rgb_array = rgb_array[:, :, :3]
        return rgb_array

    def close(self):
        p.disconnect()

In [5]:
env = BikeEnv()
np.random.seed(123)
env.seed(123)

C:\Users\SERVER\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
nb_actions = env.action_space.n

In [7]:
def create_mlp():
    inputs = Input(shape=env.observation_space['sensor'].shape)
    x = Dense(16, activation='relu')(inputs)
    x = Dense(16, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    #x = Dense(8,  activation='relu')(x)
    x = Dense(nb_actions, activation='linear')(x)
    model = Model(inputs, x)
    return model

In [8]:
def create_cnn():
    inputShape = (64, 64,1)
    inputs = Input(shape=inputShape,name='image')
    x = Conv2D(16, (3, 3), padding="same")(inputs)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(16, (3, 3), padding="same")(inputs)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(32, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(64, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(128, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Flatten()(x)
    x = Dense(32)(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(16)(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(8)(x)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(6)(x)
    x = Activation("linear")(x)
 
    model = Model(inputs, x)
    return model

In [9]:
mlp = create_mlp()
cnn = create_cnn()
combinedInput = concatenate([mlp.output, cnn.output])
x = Dense(8, activation="relu")(combinedInput)
x = Dense(nb_actions, activation="relu")(x)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
image (InputLayer)              (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 64, 64, 16)   160         image[0][0]                      
__________________________________________________________________________________________________
activation_2 (Activation)       (None, 64, 64, 16)   0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 64, 64, 16)   64          activation_2[0][0]               
___________________________________________________________________________________________

In [10]:
class CustomCalback(Callback ):
    def on_action_begin(self, action, logs={}):
        p.setRealTimeSimulation(1)

    def on_action_end(self, action, logs={}):
        p.setRealTimeSimulation(0)

In [11]:
class CustomProcessor(Processor):

    def process_state_batch(self, batch):
        batch = np.squeeze(batch, axis=1)
        b_shape = batch.shape[0]
        sensor = [batch[i][0] for i in range(b_shape)]
        images = [batch[i][1] for i in range(b_shape)]
        #return [sensor,images]

        return batch
    def process_observation(self, observation):
        
        #observation = observation['sensor'],observation['image'].reshape((64,64,1))
        observation = observation['sensor'] 
        return observation

In [12]:
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=create_mlp(), nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.processor = CustomProcessor()
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [13]:
dqn.fit(env, nb_steps=250000, visualize=False, verbose=1,callbacks  = [CustomCalback()])

Training for 250000 steps ...
Interval 1 (0 steps performed)

   11/10000 [..............................] - ETA: 1:54:16 - reward: -27.3079

C:\Users\SERVER\Anaconda3\lib\site-packages\rl\memory.py:39: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 7811s 781ms/step - reward: -29.8715
3030 episodes - episode_reward: -98.587 [-107.143, -93.421] - loss: 34.242 - mae: 80.054 - mean_q: -93.493

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 7559s 756ms/step - reward: -29.7797
3033 episodes - episode_reward: -98.185 [-106.280, -93.102] - loss: 0.855 - mae: 81.822 - mean_q: -96.241

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 7498s 750ms/step - reward: -29.9241
3054 episodes - episode_reward: -97.983 [-107.188, -92.903] - loss: 0.588 - mae: 81.847 - mean_q: -96.226

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 7621s 762ms/step - reward: -28.4265
2913 episodes - episode_reward: -97.585 [-105.591, -92.892] - loss: 0.478 - mae: 81.863 - mean_q: -96.040

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 7504s 750ms/step - reward: -27.2960
2807 episodes - 